In [1]:
!cat .env

STAGE=ENGIE_PRODUCTION


In [2]:
Company.objects.all()

<QuerySet [<Company: a17b7aca-5552-4f17-9ba0-9946029390a6: POC-Projeto APR>, <Company: 571581e3-da3a-469e-8676-0dc58c056ef7: GSO - UHE-SALTO OSORIO>, <Company: 4920816b-507a-48b1-86d5-d2eef6c63a1f: GSO - UTPS>, <Company: cff59760-69dd-4d07-bba5-61aa8b6faeea: GES - UHE-CANA BRAVA>, <Company: 90041917-e41e-46f2-b979-7d3ad53efad2: Engie Umburanas>, <Company: 69c4238a-fc42-4a16-a312-364ff4cd10c6: GSB - UHE-ITA>, <Company: 4b4bfdad-f8b8-403b-93a6-26e846a6770b: GSO - UHE-MACHADINHO>, <Company: 1b1511d1-6f28-4aed-b02c-7c820c5f9dbb: GSO - PCH-RONDONÓPOLIS>, <Company: e9fce210-f652-4d7d-befe-32e02a8f0f60: GSO - UHE-ITA>, <Company: 6ea3a49c-55a7-4aeb-919f-949034a5d2fa: GSB - UHE-ESTREITO>, <Company: e2d725b7-1b3b-4138-b790-cf49a6ab5284: GSB - UHE-JAGUARA>, <Company: 6af632a4-8b09-4bc2-bc6c-43d9a7dc23f4: GSB - UHE-MIRANDA>, <Company: 76ddb208-b049-4022-9736-9fd0101a5392: GSB - UHE-PONTE DE PEDRA>, <Company: 247c952a-9371-483d-8039-9a65b41b8db3: GSB - UHE-SÃO SALVADOR>, <Company: 24072e01-a4a7-477

In [3]:
import json
from django.contrib.admin.utils import flatten
from helpers.strings import keys_to_snake_case, keys_to_camel_case
from rest_framework_json_api.utils import format_field_names
from tqdm.notebook import tqdm

In [4]:
def generate_select_multiple_export(field):

    result={"cat":[]}    
    values=field["selectOptions"]["options"]

    for value in values:
        if field["apiName"] == "demandOrigin":
            result["cat"].append({"if":[{"in": 
                                     [value["value"], {"var": "formData."+field["apiName"]}]
                                    }, value["name"] + " ", ""]})
        else:
            result["cat"].append({"if":[{"in": 
                                     [value["value"], {"var": "formData."+field["apiName"]}]
                                    }, value["name"] + ", ", ""]})


    return result

In [5]:
def generate_select_export(field):
    result={"if":[]}    
    values=field["selectOptions"]["options"]
    
    for value in values:
        result["if"].append({"==":[{"var": "formData."+field["apiName"]}, value["value"]]})
        result["if"].append(value["name"])
        
    return result

In [16]:
def generate_export(field):
    result={"header": field["displayName"], "key": field["apiName"]}
    
    if field["dataType"]=="timestamp":
        result["isDate"]=True
    elif (field["dataType"]=="select") and ("options" in field["selectOptions"]):
        result["logic"]=generate_select_export(field)
    elif (field["dataType"]=="selectMultiple") and ("options" in field["selectOptions"]):
        result["logic"]=generate_select_multiple_export(field)
    elif field["dataType"]=="jsonLogic":
        result["logic"]=field["logic"]
    elif field["dataType"]=="float":
        result["style"]= {
                "num_fmt": "0.000"
                }
    elif field["dataType"]=="boolean":
        result["logic"]= {"var": "formData."+field["apiName"]}
    
    
    return result

In [7]:
def generate_array_export(field, number=8):
    result_list=[]
    values=field["innerFields"]
    
    for value in values:
        
        if (value["dataType"] == "select") and ("options" in value["selectOptions"]):            
            for index in range(1, number+1):            
                result={}
                result["header"]=field["displayName"] + " - " + value["displayName"] + " "  + str(index)
                result["key"]=value["apiName"]+str(index)
                result["logic"]={}
                result["logic"]["if"]=[]
                values_select=value["selectOptions"]["options"]

                for value_select in values_select:
                    result["logic"]["if"].append({"==":[{"var": "formData." + field["apiName"]+ "."+str(index-1) + "." +value["apiName"]}, value_select["value"]]})
                    result["logic"]["if"].append(value_select["name"])

                result_list.append(result)
        
        elif (value["dataType"] == "boolean") and ("display" in value["apiName"]):
            pass
        
        elif value["dataType"] == "innerImagesArray":
            pass
        
        elif (value["dataType"] == "selectMultiple") and ("options" in value["selectOptions"]):
            for index in range(1, number+1):            
                result={}
                result["header"]=field["displayName"] + " - " + value["displayName"] + " "  + str(index)
                result["key"]=value["apiName"]+str(index)
                result["logic"]={}
                result["logic"]["cat"]=[]
                values_select=value["selectOptions"]["options"]

                for value_select in values_select:
                    result["logic"]["cat"].append({"if":[{"in": 
                                     [value_select["value"], {"var": "formData." + field["apiName"]+ "."+str(index-1) + "." +value["apiName"]}]
                                    }, value_select["name"], ", "]})

                result_list.append(result)
            
        else:
            for index in range(1, number+1):
                result={}
                result["header"]=field["displayName"] + " - " + value["displayName"] + " " + str(index)
                result["key"]=value["apiName"]+str(index)
                result["logic"]= {"var": "formData." + field["apiName"] + "." + str(index-1) + "." + value["apiName"]}
                result_list.append(result)

    
    
    return result_list

In [8]:
def update_export_data(data, occ=None, filename=None):
    if occ is not None:
        if 'measurement_columns' in occ.form_fields.keys():
            del occ.form_fields['measurement_columns']
        occ.form_fields['measurementColumns']=data
        occ.save()
    if filename is not None:
        with open(filename + '.txt', 'w') as outfile:
            json.dump(data, outfile, indent=4)

In [9]:
def format_keys(obj, format_type="camelize"):
    if isinstance(obj, dict):
        for key, value in obj.items():
            if isinstance(value, dict):
                obj[key] = format_field_names(
                            format_keys(value), format_type
                            )
            elif isinstance(value, list):
                list_temp = value.copy()
                for i, item in enumerate(value):
                    if isinstance(item, dict):
                        list_temp[i] = format_field_names(
                        format_keys(item), format_type
                        )
                    else:
                        list_temp[i] = item
                obj[key] = list_temp
    return obj

In [10]:
company=Company.objects.get(name='GSO - UHE-ESTREITO')
company

<Company: be176150-b4da-4162-9eec-67f6db3f5612: CCR - RodoAnel>

In [13]:
# occ=OccurrenceType.objects.get(uuid='f6033d7d-9649-420d-9125-2000aefcd25a')
occ=OccurrenceType.objects.filter(name='Edificação sociofamiliar',company__name__startswith='GSO').order_by('name').distinct()
occ.count(),occ

(1,
 <QuerySet [<OccurrenceType: Edificação sociofamiliar - ['GSO - UHE-ITA', 'GSO - UHE-SÃO SALVADOR', 'GSO - UHE-MACHADINHO', 'GSO - PCH-RONDONÓPOLIS', 'GSO - UHE-PASSO FUNDO', 'GSO - UHE-SALTO SANTIAGO', 'GSO - UHE-MIRANDA', 'GSO - UHE-SALTO OSORIO', 'GSO - UHE-CANA BRAVA', 'GSO - UHE-JAGUARA', 'GSO - UTPS', 'GSO - UHE-PONTE DE PEDRA', 'GSO - PCH-JOSÉ GELÁZIO DA ROCHA', 'GSO - UHE-ESTREITO']>]>)

In [12]:
# for match in tqdm(occ):
#     for index, field in enumerate(match.form_fields['fields']):
#         match.form_fields['fields'][index]=format_keys(keys_to_camel_case(field))
        
#     match.save()

  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
# columns=[]

# for field in occ.form_fields['fields']:
    
#     if field['dataType'] != 'arrayOfObjects':
#         columns.append(generate_export(field))
#     else:
#         columns.append(generate_array_export(field))
    

# columns=flatten(columns)

# update_export_data(columns, occ=occ)

In [17]:
for match in tqdm(occ):
        
    try:
        columns=[]

        for field in match.form_fields['fields']:
            if field['dataType'] != 'arrayOfObjects':
                columns.append(generate_export(field))
            else:
#                 pass
                columns.append(generate_array_export(field, 5))

        columns=flatten(columns)
#         print(columns)
#         update_export_data(columns, occ=match)
    
    except Exception as e:
        print(f'{match.uuid} - {e}')

  0%|          | 0/1 [00:00<?, ?it/s]

f36530bb-d8f9-4910-be86-7d4ca324539d - 'dataType'


In [18]:
columns

[[{'header': 'Restrições - Existem restrições? 1',
   'key': 'registrationProhibited1',
   'logic': {'if': [{'==': [{'var': 'formData.restriction.0.registrationProhibited'},
       '1']},
     'Sim',
     {'==': [{'var': 'formData.restriction.0.registrationProhibited'}, '2']},
     'Não']}},
  {'header': 'Restrições - Existem restrições? 2',
   'key': 'registrationProhibited2',
   'logic': {'if': [{'==': [{'var': 'formData.restriction.1.registrationProhibited'},
       '1']},
     'Sim',
     {'==': [{'var': 'formData.restriction.1.registrationProhibited'}, '2']},
     'Não']}},
  {'header': 'Restrições - Existem restrições? 3',
   'key': 'registrationProhibited3',
   'logic': {'if': [{'==': [{'var': 'formData.restriction.2.registrationProhibited'},
       '1']},
     'Sim',
     {'==': [{'var': 'formData.restriction.2.registrationProhibited'}, '2']},
     'Não']}},
  {'header': 'Restrições - Existem restrições? 4',
   'key': 'registrationProhibited4',
   'logic': {'if': [{'==': [{'var

# Verifica se o campo existe na exportação da API

In [16]:
for match in tqdm(occ):
        
    try:
        columns=[]

        for field in match.form_fields['fields']:
            if field['dataType'] != 'arrayOfObjects':
                columns.append(generate_export(field))
            else:
#                 pass
                columns.append(generate_array_export(field, 5))

#         columns=flatten(columns)
#         print(columns)
#         update_export_data(columns, occ=match)
    
    except Exception as e:
        print(f'{match.uuid} - {e}')

  0%|          | 0/4 [00:00<?, ?it/s]

In [28]:
keys=[]
for a in columns:
    try:
        if type(a) is not list:
            keys.append(a['key'])
            print(a['key'])
        elif a != []:
            keys.extend([b['key'] for b in a])
            print([b['key'] for b in a])
    except Exception as e:
        print(e,type(a),a)

idCcrAntt
altura
largura
extensao
latidudeini
longitudeini
xini
yini
latitudefim
longitudefim
xfim
yfim
zona
tipoelemento
material
ambiente
detalheelemento
marginal
inspectionCampaignYear
generalConservationState
observacoesconservacao
reparar
extensaoreparo
limpeza
extensaolimpeza
rocada
extensaorocada
implantar
extensaoimplantacao
['localanomalia1', 'localanomalia2', 'localanomalia3', 'localanomalia4', 'localanomalia5', 'anomaly1', 'anomaly2', 'anomaly3', 'anomaly4', 'anomaly5', 'occurrenceType1', 'occurrenceType2', 'occurrenceType3', 'occurrenceType4', 'occurrenceType5', 'description1', 'description2', 'description3', 'description4', 'description5']
observacoes
inspectionCampaignSemester
drenagemTerraplenos
recuperationSixty
recuperationNinety


In [29]:
keys

['idCcrAntt',
 'altura',
 'largura',
 'extensao',
 'latidudeini',
 'longitudeini',
 'xini',
 'yini',
 'latitudefim',
 'longitudefim',
 'xfim',
 'yfim',
 'zona',
 'tipoelemento',
 'material',
 'ambiente',
 'detalheelemento',
 'marginal',
 'inspectionCampaignYear',
 'generalConservationState',
 'observacoesconservacao',
 'reparar',
 'extensaoreparo',
 'limpeza',
 'extensaolimpeza',
 'rocada',
 'extensaorocada',
 'implantar',
 'extensaoimplantacao',
 'localanomalia1',
 'localanomalia2',
 'localanomalia3',
 'localanomalia4',
 'localanomalia5',
 'anomaly1',
 'anomaly2',
 'anomaly3',
 'anomaly4',
 'anomaly5',
 'occurrenceType1',
 'occurrenceType2',
 'occurrenceType3',
 'occurrenceType4',
 'occurrenceType5',
 'description1',
 'description2',
 'description3',
 'description4',
 'description5',
 'observacoes',
 'inspectionCampaignSemester',
 'drenagemTerraplenos',
 'recuperationSixty',
 'recuperationNinety']

In [43]:
def filtrar_por_chave(lista_de_dicionarios, valor):
    try:
        for item in lista_de_dicionarios:
            if isinstance(item, dict):
                if item.get('key') == valor:
                    return item
            elif isinstance(item, list):
                for innerItem in item:
                    if innerItem.get('key') == valor:
                        return innerItem
    except:
        pass

In [45]:
filtrar_por_chave(columns,'anomaly3')

{'header': 'Anomalias - Anomalia 3',
 'key': 'anomaly3',
 'logic': {'if': [{'==': [{'var': 'formData.therapy.2.anomaly'}, '1']},
   'Assoreado/obstruído\xa0',
   {'==': [{'var': 'formData.therapy.2.anomaly'}, '2']},
   'Ausência (projeto de melhoria/ implantação)\xa0',
   {'==': [{'var': 'formData.therapy.2.anomaly'}, '3']},
   'Elemento danificado/desalinhado/infiltração nas juntas\xa0',
   {'==': [{'var': 'formData.therapy.2.anomaly'}, '4']},
   'Elemento danificado (grelha/tampa/parede)\xa0',
   {'==': [{'var': 'formData.therapy.2.anomaly'}, '5']},
   'Ausência (grelha/tampa)\xa0',
   {'==': [{'var': 'formData.therapy.2.anomaly'}, '6']},
   'Sujo/Afogado/obstruído/retenção de resíduos\xa0',
   {'==': [{'var': 'formData.therapy.2.anomaly'}, '7']},
   'Erosão\xa0',
   {'==': [{'var': 'formData.therapy.2.anomaly'}, '8']},
   'Densa vegetação no entorno\xa0',
   {'==': [{'var': 'formData.therapy.2.anomaly'}, '9']},
   'Elemento praguejado\xa0']}}

In [46]:
report=[]
for a in keys:
    if a not in [b['key'] for b in company.custom_options['reportingSpreadsheet']['extraColumns']]:
        report.append(filtrar_por_chave(columns,a))

In [47]:
report

[{'header': 'Identificação Poder Regulador', 'key': 'idCcrAntt'},
 {'header': 'Altura', 'key': 'altura', 'style': {'num_fmt': '0.000'}},
 {'header': 'Largura', 'key': 'largura', 'style': {'num_fmt': '0.000'}},
 {'header': 'Extensão', 'key': 'extensao', 'style': {'num_fmt': '0.000'}},
 {'header': 'Latitude Início',
  'key': 'latidudeini',
  'style': {'num_fmt': '0.000'}},
 {'header': 'Longitude Início',
  'key': 'longitudeini',
  'style': {'num_fmt': '0.000'}},
 {'header': 'X Inicio (UTM)', 'key': 'xini', 'style': {'num_fmt': '0.000'}},
 {'header': 'Y Inicio (UTM)', 'key': 'yini', 'style': {'num_fmt': '0.000'}},
 {'header': 'Latitude Fim',
  'key': 'latitudefim',
  'style': {'num_fmt': '0.000'}},
 {'header': 'Longitude Fim',
  'key': 'longitudefim',
  'style': {'num_fmt': '0.000'}},
 {'header': 'X Fim (UTM)', 'key': 'xfim', 'style': {'num_fmt': '0.000'}},
 {'header': 'Y Fim (UTM)', 'key': 'yfim', 'style': {'num_fmt': '0.000'}},
 {'header': 'Zona (UTM)', 'key': 'zona'},
 {'header': 'Deta